# Re-Imaging Price Trends - Data Generation

**Purpose**: Generate WRDS CRSP stock price data for paper reproduction

**Output**: 
- `data_1993_2000_train_val.parquet` - Training/validation data
- `data_2001_2019_test.parquet` - Test data

**Next Step**: Run image generation scripts after completion

## 1. Environment Setup

In [ ]:
# Install required packages
!pip install wrds

import wrds
import pandas as pd
import numpy as np
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("Environment setup completed!")

## 2. WRDS CRSP Data Collection

In [ ]:
# WRDS Connection
print("🔗 Connecting to WRDS...")
db = wrds.Connection()
print("✅ Connected successfully!")

In [ ]:
# CRSP Data Query - Full Dataset (1992-2019)
query = """
    SELECT
        a.permno AS code,
        a.date,
        a.prc AS close,
        a.vol AS volume,
        a.ret,          -- returns
        a.openprc AS open,
        a.askhi AS high,
        a.bidlo AS low,
        a.shrout,       -- shares outstanding (for market cap)
        b.ticker
    FROM
        crsp.dsf AS a
    LEFT JOIN
        crsp.dsenames AS b
        ON a.permno = b.permno AND b.namedt <= a.date AND a.date <= b.nameendt
    WHERE
        a.date BETWEEN '1992-01-01' AND '2019-12-31'
        AND b.shrcd IN (10, 11)  -- Common stocks only
        AND b.exchcd IN (1, 2, 3)  -- NYSE, AMEX, NASDAQ
"""

print("📥 Downloading CRSP data (1992-2019)... (This may take several minutes)")
df_raw = db.raw_sql(query, date_cols=['date'])
db.close()

print(f"✅ Download completed!")
print(f"📊 Raw data: {len(df_raw):,} records")
print(f"🏢 Unique stocks: {df_raw['code'].nunique():,}")
print(f"📅 Period: {df_raw['date'].min()} ~ {df_raw['date'].max()}")

## 3. Data Preprocessing

In [ ]:
# Basic Data Analysis
print("📊 RAW DATA ANALYSIS")
print("=" * 50)

print(f"Total records: {len(df_raw):,}")
print(f"Unique companies: {df_raw['code'].nunique():,}")
print(f"Date range: {df_raw['date'].min()} to {df_raw['date'].max()}")
print(f"Trading days: {df_raw['date'].nunique():,}")

# Missing values
missing_stats = df_raw.isnull().sum()
for col, missing_count in missing_stats.items():
    if missing_count > 0:
        pct = missing_count / len(df_raw) * 100
        print(f"Missing {col}: {missing_count:,} ({pct:.1f}%)")

In [ ]:
# Data Preprocessing (equity_data.py 방식 적용)
df = df_raw.copy()

# 1. 컬럼명 변경 (equity_data.py 방식)
df = df.rename(columns={
    'code': 'StockID',      # WRDS의 code → equity_data의 StockID
    'date': 'Date',         # 동일
    'close': 'Close',       # 동일  
    'volume': 'Vol',        # WRDS의 volume → equity_data의 Vol
    'ret': 'Ret',          # 동일
    'open': 'Open',        # 동일
    'high': 'High',        # 동일
    'low': 'Low',          # 동일
    'shrout': 'Shares',    # WRDS의 shrout → equity_data의 Shares
})

# 2. 데이터 타입 변환 (equity_data.py 방식)
df.StockID = df.StockID.astype(str)
df.Ret = df.Ret.astype(str)

# 3. 무효한 값들 NaN으로 처리 (equity_data.py 방식)
df = df.replace({
    "Close": {0: np.nan},   
    "Open": {0: np.nan},
    "High": {0: np.nan},
    "Low": {0: np.nan},
    "Ret": {"C": np.nan, "B": np.nan, "A": np.nan, ".": np.nan},
    "Vol": {0: np.nan, (-99): np.nan},
})

# 4. 수익률을 numeric으로 변환하고 NaN 제거
df["Ret"] = df.Ret.astype(np.float64)
df = df.dropna(subset=["Ret"])

# 5. 절댓값 처리 (CRSP 음수 가격 처리)
df[["Close", "Open", "High", "Low", "Vol", "Shares"]] = df[
    ["Close", "Open", "High", "Low", "Vol", "Shares"]
].abs()

print("Data preprocessing (equity_data.py style) completed")

In [ ]:
# Market Cap, MultiIndex, Log Returns (equity_data.py 방식)
print("\nCalculating market cap and setting up data structure...")

# 1. 시장캡 계산 (equity_data.py 방식)
df["MarketCap"] = np.abs(df["Close"] * df["Shares"])

# 2. MultiIndex 설정 (equity_data.py 방식)
df.set_index(["Date", "StockID"], inplace=True)
df.sort_index(inplace=True)

# 3. 로그 수익률 및 누적 로그 수익률 계산 (equity_data.py 방식)
df["log_ret"] = np.log(1 + df.Ret)
df["cum_log_ret"] = df.groupby("StockID")["log_ret"].cumsum(skipna=True)

# 4. EWMA 변동성 (equity_data.py와 정확히 동일)
print("Calculating EWMA volatility...")
df["EWMA_vol"] = df.groupby("StockID")["Ret"].transform(
    lambda x: (x**2).ewm(alpha=0.05).mean().shift(periods=1)  # 1-day lag
)

print("Data structure setup completed (equity_data.py style)")

## 4. Future Returns & Labels

In [ ]:
# Future Returns Calculation (equity_data.py 방식 - 핵심만)
print("Computing future returns using cumulative log returns...")

# equity_data.py Lines 214-218: 고정 기간 수익률만 계산 (이미지 생성용)
for i in [5, 20, 60, 65, 180, 250, 260]:
    print(f"Calculating {i}d return")
    df[f"Ret_{i}d"] = df.groupby("StockID")["cum_log_ret"].apply(
        lambda x: np.exp(x.shift(-i) - x) - 1  # equity_data.py와 정확히 동일한 공식
    )

# 백분율로 변환 (dataset.py 호환성)
df['ret5'] = df['Ret_5d'] * 100   
df['ret20'] = df['Ret_20d'] * 100
df['ret60'] = df['Ret_60d'] * 100

# Binary labels
df['label_5'] = np.where(df['Ret_5d'] > 0, 1, 0)
df['label_20'] = np.where(df['Ret_20d'] > 0, 1, 0) 
df['label_60'] = np.where(df['Ret_60d'] > 0, 1, 0)

print("Future returns calculated (equity_data.py style)")
print("Note: Period-based returns (week/month/quarter) skipped - for portfolio rebalancing later")

In [ ]:
# Data Quality Check (MultiIndex 구조에 맞게 수정)
print("\nData Quality Summary:")
print(f"Total records: {len(df):,}")
print(f"Unique companies: {df.index.get_level_values('StockID').nunique():,}")
print(f"Date range: {df.index.get_level_values('Date').min()} to {df.index.get_level_values('Date').max()}")

# Check future returns availability
for period in [5, 20, 60]:
    ret_col = f'Ret_{period}d'
    valid_count = df[ret_col].notna().sum()
    up_count = df[f'label_{period}'].sum()
    up_pct = up_count / valid_count * 100 if valid_count > 0 else 0
    print(f"{period}-day returns: {valid_count:,} valid ({up_pct:.1f}% up)")

print("\nData ready for image generation (equity_data.py style)")

In [ ]:
# Sample of final data (MultiIndex 구조)
print("\nSample of processed data:")
sample_cols = ['Close', 'Vol', 'Ret', 'MarketCap', 'EWMA_vol', 
               'Ret_5d', 'Ret_20d', 'Ret_60d', 'ret5', 'ret20', 'ret60', 
               'label_5', 'label_20', 'label_60']

# MultiIndex 구조에서 샘플 보기
print("First 10 records:")
print(df[sample_cols].head(10).to_string())

print("\nIndex structure:")
print(f"Date index: {df.index.get_level_values('Date')[:5].tolist()}")  
print(f"StockID index: {df.index.get_level_values('StockID')[:5].tolist()}")

## 5. Save Final Datasets

In [ ]:
# Save unified dataset (1992-2019 전체 통합)
print("Saving unified dataset (1992-2019, equity_data.py processed)...")

# MultiIndex를 reset해서 일반 DataFrame으로 변환
df_save = df.reset_index()

# 컬럼명을 dataset.py가 기대하는 형태로 변환
df_save = df_save.rename(columns={
    'StockID': 'code',        
    'Date': 'date',           
    'Close': 'close',         
    'Vol': 'volume',          
    'Ret': 'ret',            
    'Open': 'open',          
    'High': 'high',          
    'Low': 'low',            
    'MarketCap': 'mktcap',   
    'EWMA_vol': 'ewma_vol'   
})

# 하나의 통합 파일로 저장 (분할 없음)
unified_filename = 'data_1992_2019_unified.parquet'
df_save.to_parquet(unified_filename, index=False)

print(f"✅ Unified dataset: {unified_filename}")
print(f"   Records: {len(df_save):,}")
print(f"   Companies: {df_save['code'].nunique():,}")
print(f"   Period: {df_save['date'].min()} ~ {df_save['date'].max()}")

print(f"\n🎉 Data generation completed!")
print(f"   → Single unified file with equity_data.py processing")
print(f"   → Ready for image generation with proper lookback data")

In [ ]:
# Summary Statistics for Paper
print("📋 SUMMARY STATISTICS FOR PAPER")
print("=" * 60)

print(f"\n🔍 Dataset Summary:")
print(f"   • Period: 1992-2019 ({df_save['year'].nunique()} years)")
print(f"   • Total observations: {len(df_save):,}")
print(f"   • Unique companies: {df_save['code'].nunique():,}")
print(f"   • Average company coverage: {len(df_save)/df_save['code'].nunique():.0f} obs/company")

print(f"\n📊 Label Distribution (Up/Down Movements):")
for period in [5, 20, 60]:
    label_col = f'label_{period}'
    valid_labels = df_save[label_col].dropna()
    up_count = (valid_labels == 1).sum()
    total_count = len(valid_labels)
    up_pct = up_count / total_count * 100
    
    print(f"   • {period}-day: {up_count:,}/{total_count:,} up ({up_pct:.1f}% up, {100-up_pct:.1f}% down)")

print(f"\n💰 Market Cap Distribution:")
valid_mktcap = df_save['mktcap'].dropna()
percentiles = valid_mktcap.quantile([0.1, 0.25, 0.5, 0.75, 0.9])
print(f"   • 10th percentile: ${percentiles[0.1]/1e6:.0f}M")
print(f"   • 25th percentile: ${percentiles[0.25]/1e6:.0f}M") 
print(f"   • Median: ${percentiles[0.5]/1e6:.0f}M")
print(f"   • 75th percentile: ${percentiles[0.75]/1e6:.0f}M")
print(f"   • 90th percentile: ${percentiles[0.9]/1e6:.0f}M")

print(f"\n📈 Return Characteristics:")
print(f"   • Daily return volatility: {df_save['ret'].std()*100:.2f}% per day")
print(f"   • 5-day return volatility: {df_save['ret5'].std():.2f}%")
print(f"   • 20-day return volatility: {df_save['ret20'].std():.2f}%")  
print(f"   • 60-day return volatility: {df_save['ret60'].std():.2f}%")

print(f"\n✅ Data ready for CNN image generation and training!")
print(f"   → equity_data.py processing: ✅ Complete")
print(f"   → Cumulative log return calculation: ✅ Complete") 
print(f"   → Future return labels: ✅ Complete")
print(f"   → Missing value handling: ✅ Complete")
print(f"   → 1992 lookback data: ✅ Available for moving averages")

In [ ]:
# Visualization: Distribution Plots
print("📊 CREATING DISTRIBUTION PLOTS")
print("=" * 60)

# Set up the plotting style
plt.style.use('default')
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Stock Data Distribution Analysis (1992-2019)', fontsize=16, fontweight='bold')

# 1. Market Cap Distribution (Log Scale)
valid_mktcap = df_save['mktcap'].dropna()
axes[0,0].hist(np.log10(valid_mktcap), bins=50, alpha=0.7, color='skyblue', edgecolor='black')
axes[0,0].set_title('Market Cap Distribution (Log10)', fontweight='bold')
axes[0,0].set_xlabel('Log10(Market Cap $)')
axes[0,0].set_ylabel('Frequency')
axes[0,0].grid(True, alpha=0.3)

# 2. Daily Returns Distribution
valid_ret = df_save['ret'].dropna()
axes[0,1].hist(valid_ret * 100, bins=100, alpha=0.7, color='lightgreen', edgecolor='black', range=(-20, 20))
axes[0,1].set_title('Daily Returns Distribution', fontweight='bold')  
axes[0,1].set_xlabel('Daily Return (%)')
axes[0,1].set_ylabel('Frequency')
axes[0,1].axvline(x=0, color='red', linestyle='--', alpha=0.8)
axes[0,1].grid(True, alpha=0.3)

# 3. Price Level Distribution (Log Scale)
valid_close = df_save['close'].dropna()
axes[0,2].hist(np.log10(valid_close), bins=50, alpha=0.7, color='orange', edgecolor='black')
axes[0,2].set_title('Stock Price Distribution (Log10)', fontweight='bold')
axes[0,2].set_xlabel('Log10(Close Price $)')
axes[0,2].set_ylabel('Frequency')
axes[0,2].grid(True, alpha=0.3)

# 4. 5-day Future Returns
valid_ret5 = df_save['ret5'].dropna()
axes[1,0].hist(valid_ret5, bins=100, alpha=0.7, color='coral', edgecolor='black', range=(-50, 50))
axes[1,0].set_title('5-Day Future Returns', fontweight='bold')
axes[1,0].set_xlabel('5-Day Return (%)')
axes[1,0].set_ylabel('Frequency')
axes[1,0].axvline(x=0, color='red', linestyle='--', alpha=0.8)
axes[1,0].grid(True, alpha=0.3)

# 5. 20-day Future Returns  
valid_ret20 = df_save['ret20'].dropna()
axes[1,1].hist(valid_ret20, bins=100, alpha=0.7, color='mediumpurple', edgecolor='black', range=(-80, 80))
axes[1,1].set_title('20-Day Future Returns', fontweight='bold')
axes[1,1].set_xlabel('20-Day Return (%)')
axes[1,1].set_ylabel('Frequency')
axes[1,1].axvline(x=0, color='red', linestyle='--', alpha=0.8)
axes[1,1].grid(True, alpha=0.3)

# 6. 60-day Future Returns
valid_ret60 = df_save['ret60'].dropna()
axes[1,2].hist(valid_ret60, bins=100, alpha=0.7, color='gold', edgecolor='black', range=(-150, 150))
axes[1,2].set_title('60-Day Future Returns', fontweight='bold')
axes[1,2].set_xlabel('60-Day Return (%)')
axes[1,2].set_ylabel('Frequency')
axes[1,2].axvline(x=0, color='red', linestyle='--', alpha=0.8)
axes[1,2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('data_distribution_analysis.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

print("✅ Distribution plots saved as: data_distribution_analysis.png")

In [ ]:
# Time Series Analysis
print("📅 TIME SERIES ANALYSIS")
print("=" * 60)

# Returns by year
print(f"\n1. Annual Statistics:")
df_save['year'] = pd.to_datetime(df_save['date']).dt.year

for year in sorted(df_save['year'].unique()):
    year_data = df_save[df_save['year'] == year]
    
    # Count data
    total_obs = len(year_data)
    total_companies = year_data['code'].nunique()
    
    # Daily returns
    year_ret = year_data['ret'].dropna()
    mean_daily_ret = year_ret.mean() * 100
    
    # 5d future returns  
    year_ret5 = year_data['ret5'].dropna()
    up_5d = (year_data['label_5'] == 1).sum()
    total_5d = year_data['label_5'].notna().sum()
    up_pct_5d = up_5d / total_5d * 100 if total_5d > 0 else 0
    
    print(f"   {year}: {total_obs:,} obs, {total_companies:,} companies, "
          f"daily_ret={mean_daily_ret:.3f}%, 5d_up={up_pct_5d:.1f}%")

# Market cap deciles analysis  
print(f"\n2. Market Cap Decile Analysis (most recent year):")
recent_year = df_save['year'].max()
recent_data = df_save[df_save['year'] == recent_year].copy()

# Create market cap deciles
recent_data = recent_data.dropna(subset=['mktcap'])
if len(recent_data) > 0:
    recent_data['mktcap_decile'] = pd.qcut(recent_data['mktcap'], 
                                          q=10, labels=False, duplicates='drop') + 1
    
    print(f"   Based on {recent_year} data:")
    for decile in sorted(recent_data['mktcap_decile'].unique()):
        decile_data = recent_data[recent_data['mktcap_decile'] == decile]
        
        # Market cap stats
        mean_mktcap = decile_data['mktcap'].mean() / 1e6
        median_mktcap = decile_data['mktcap'].median() / 1e6
        
        # Return stats
        mean_ret = decile_data['ret'].mean() * 100
        up_5d = (decile_data['label_5'] == 1).sum()
        total_5d = decile_data['label_5'].notna().sum()
        up_pct_5d = up_5d / total_5d * 100 if total_5d > 0 else 0
        
        print(f"     Decile {decile:2.0f}: MktCap=${median_mktcap:6.0f}M, "
              f"DailyRet={mean_ret:.3f}%, 5d_Up={up_pct_5d:.1f}%")

In [ ]:
# Return Distribution Analysis (핵심 분석)
print("📈 RETURN DISTRIBUTION ANALYSIS")
print("=" * 60)

# Daily returns
print(f"\n1. Daily Returns (ret):")
valid_ret = df_save['ret'].dropna()
positive_ret = (valid_ret > 0).sum()
negative_ret = (valid_ret < 0).sum()
zero_ret = (valid_ret == 0).sum()
total_ret = len(valid_ret)

print(f"   Total observations: {total_ret:,}")
print(f"   Positive returns: {positive_ret:,} ({positive_ret/total_ret*100:.1f}%)")
print(f"   Negative returns: {negative_ret:,} ({negative_ret/total_ret*100:.1f}%)")
print(f"   Zero returns: {zero_ret:,} ({zero_ret/total_ret*100:.1f}%)")
print(f"   Mean: {valid_ret.mean()*100:.3f}% per day")
print(f"   Std: {valid_ret.std()*100:.2f}% per day")

# Future returns analysis
for period in [5, 20, 60]:
    print(f"\n{period+1}. {period}-day Future Returns:")
    
    # Percentage returns
    ret_col = f'ret{period}'
    label_col = f'label_{period}'
    
    valid_fut_ret = df_save[ret_col].dropna()
    valid_labels = df_save[label_col].dropna()
    
    up_count = (valid_labels == 1).sum()
    down_count = (valid_labels == 0).sum()
    total_labels = len(valid_labels)
    
    print(f"   Valid observations: {len(valid_fut_ret):,}")
    print(f"   Up movements (>0%): {up_count:,} ({up_count/total_labels*100:.1f}%)")
    print(f"   Down movements (≤0%): {down_count:,} ({down_count/total_labels*100:.1f}%)")
    print(f"   Mean return: {valid_fut_ret.mean():.2f}%")
    print(f"   Std return: {valid_fut_ret.std():.2f}%")
    print(f"   Min return: {valid_fut_ret.min():.2f}%")
    print(f"   Max return: {valid_fut_ret.max():.2f}%")
    
    # Percentiles
    percentiles = valid_fut_ret.quantile([0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99])
    print(f"   Percentiles: 1%={percentiles[0.01]:.1f}%, 5%={percentiles[0.05]:.1f}%, 25%={percentiles[0.25]:.1f}%, 50%={percentiles[0.5]:.1f}%, 75%={percentiles[0.75]:.1f}%, 95%={percentiles[0.95]:.1f}%, 99%={percentiles[0.99]:.1f}%")

In [ ]:
# Basic Data Distribution Analysis
import matplotlib.pyplot as plt
import seaborn as sns

print("📊 DATA DISTRIBUTION ANALYSIS")
print("=" * 60)

# Basic statistics
print(f"\n1. Dataset Overview:")
print(f"   Total records: {len(df_save):,}")
print(f"   Unique companies: {df_save['code'].nunique():,}")
print(f"   Date range: {df_save['date'].min()} ~ {df_save['date'].max()}")
print(f"   Trading days: {df_save['date'].nunique():,}")

# Market cap distribution
print(f"\n2. Market Cap Distribution:")
valid_mktcap = df_save['mktcap'].dropna()
print(f"   Mean: ${valid_mktcap.mean()/1e6:.1f}M")
print(f"   Median: ${valid_mktcap.median()/1e6:.1f}M") 
print(f"   Std: ${valid_mktcap.std()/1e6:.1f}M")
print(f"   Min: ${valid_mktcap.min()/1e6:.1f}M")
print(f"   Max: ${valid_mktcap.max()/1e6:.1f}M")

# Price distribution
print(f"\n3. Price Level Distribution:")
valid_close = df_save['close'].dropna()
print(f"   Mean Close: ${valid_close.mean():.2f}")
print(f"   Median Close: ${valid_close.median():.2f}")
print(f"   Min Close: ${valid_close.min():.2f}")
print(f"   Max Close: ${valid_close.max():.2f}")

# Volume distribution
print(f"\n4. Volume Distribution:")
valid_volume = df_save['volume'].dropna()
print(f"   Mean Volume: {valid_volume.mean()/1e6:.1f}M shares")
print(f"   Median Volume: {valid_volume.median()/1e6:.1f}M shares")

## 6. Exploratory Data Analysis (EDA)